In [2]:
import firebase_admin
from firebase_admin import credentials, firestore

# Firebase 認証情報の読み込み
cred = credentials.Certificate("../firebase/serviceAccountKey.json")
firebase_admin.initialize_app(cred)

# Firestore クライアント
db = firestore.client()


In [3]:
def get_scripts():
    # "Scripts" コレクションからデータを取得
    scripts_ref = db.collection("Scripts")
    docs = scripts_ref.stream()

    scripts_data = []
    for doc in docs:
        data = doc.to_dict()
        scripts_data.append(data.get("content", ""))  # ネタの全文を取得
    return scripts_data

# ネタデータ取得
scripts = get_scripts()

In [4]:
import random
def get_random_comedians_data():
    # "Scripts" コレクションからデータを取得
    scripts_ref = db.collection("Comedians")
    docs = list(scripts_ref.stream())
    random_docs = random.sample(docs, 2) if len(docs) >= 2 else docs
    scripts_data = [doc.to_dict() for doc in random_docs]
    print(scripts_data)  # 確認用
    return scripts_data

# ネタデータ取得
comedians = get_random_comedians_data()

[{'gender': 'male', 'skills': {'writing_skill': 4, 'specialty_topics': 'テレビゲーム、音楽鑑賞、映画鑑賞', 'voice_characteristics': '特に記載なし', 'role': 'ボケ'}, 'birthdate': '1994-05-02', 'agency': 'プロ（吉本興業）', 'name': 'けんしろを'}, {'gender': 'male', 'skills': {'writing_skill': 3, 'specialty_topics': '飲酒、競馬、散歩、美川憲一のものまね', 'voice_characteristics': '特に記載なし', 'role': 'ツッコミ'}, 'birthdate': '1985-08-12', 'agency': 'プロ（吉本興業）', 'name': '広木英介'}]


In [6]:
print(scripts)

[' [音楽] 終わらせましょうお前には無理なお前には (01:14) その権限ないからやめてくださいはい真空 女子ですよろしくお願いし ますありがとうございますいやあの今一番 求められてるのってやっぱ商店街のロケだ なと思うんでね確かにね商店街のロケはめ てかもねいや今一番求められてるのは 子育て支援だろお前がちゃんと否定しろ こういう時 は本日はこちらの商店街を紹介していき たいと思いますよろしくお願いしますどう も会長の木野まですセーラージュピターと 同じ名前だよろしくお願いしますまこ ちゃんあありがとうございますこちらの 商店街はどんな特徴があるんですかそう ですね偏った政党のポスターが多いそう いうことじゃなくてもうちょっと ポジティブな特徴ないですかあとお客様 アンケートとって入り口から人気の店順に 並んでますお少年ジャンプの掲載順と同じ だなんで出口の方に行くと瞑想しちゃっ てるお店が増えてそこまで同じなんですね そんなにせなくていいと思いますけどま ジャンプは最後まで面白いですあそうです (02:18) よ ねご案内しますお願いしますちょちょ ちょちょ勝手にドラクエみたいにしないで ください好きですけどあこちらが私と妻が 出会った思い出のパン屋さんです思い出の パン屋さんいいですね私がパンに手を 伸ばした時に偶然妻と手と手が触れ合い ましたトングを使わない2人 が2人とももらうない手で行ったんすか 結婚指はもうこの商店街で買ったんですよ ああびっくりした薬指以外を立ててもらっ ていいですかすごい綺麗ああすいません あの横から見たら同じなんですいません 下げてくださいピント合わせないでいいん でを急急いではないです けど年寄りから多く取ろう偏った正当の ポスタ見てらんないなちょっとおらどけ どけおらどけどけあすいませんどこ見て 歩いてんだよセグウェイの暴走族 だすごい 姿勢あらティーバティーバあもうテレビ テレビじゃないんだティーバ時代を感じる (03:26) なこちら私の経営する店ですさんのお店 はい手作りの鎖を販売しておりまして個人 経営のチェーン店やしいです ねま1本1本こだわって作ってますんで じゃ結構お値段もするんじゃないですかあ まそうですねまいやらしい話本当に おっぱいが大好きで本当にらしい話あ本当 にらしい話は大丈夫です今そういうんじゃ

In [60]:
import json
import re

MAX_CHARS = 512  # 1つの `parts` に含める最大文字数
MAX_LIST_SIZE = 5  # `contents` の最大要素数

def split_text(text, max_chars):
    """ 指定の文字数でテキストを分割 """
    return [text[i:i + max_chars] for i in range(0, len(text), max_chars)]

def create_turns(text):
    """ 漫才のボケ・ツッコミを推定して交互に分割する """
    sentences = re.split(r'(?<=[。！？\n])|\s+', text)
    turns = []

    for i, sentence in enumerate(sentences):
        if sentence.strip():
            role = "user" if i % 2 == 0 else "model"  # 交互に役割を振り分け
            turns.append({"role": role, "parts": [{"text": sentence.strip()}]})

    return turns

cleaned_data = []
with open("cleaned_scripts_dataset.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        text = data.get("contents", "").strip()

        # 不要なタグやタイムスタンプを削除
        text = text.replace("[音楽]", "").strip()
        text = re.sub(r"\(\d+:\d+\)", "", text)  # (01:14) のような部分を削除

        # ボケ・ツッコミを交互に割り当てる
        formatted_contents = create_turns(text)

        if len(formatted_contents) < 2:
            continue  # user と model の両方が含まれていないデータは除外

        for i in range(0, len(formatted_contents), MAX_LIST_SIZE):
            cleaned_data.append({"contents": formatted_contents[i:i + MAX_LIST_SIZE]})

# 修正済み JSONL を保存
with open("final_scripts_dataset.jsonl", "w", encoding="utf-8") as f:
    for item in cleaned_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")


80
106
103
86
105
117
90
77
71
71


In [122]:
from vertexai.tuning import sft
import time

sft_tuning_job = sft.train(
    source_model="gemini-1.0-pro-002",
    train_dataset="gs://ai_hackathon_manzai_us_dataset/scripts_dataset.jsonl",
)

while not sft_tuning_job.has_ended:
    time.sleep(60)
    sft_tuning_job.refresh()

print(sft_tuning_job.tuned_model_name)
print(sft_tuning_job.tuned_model_endpoint_name)
print(sft_tuning_job.experiment)

Creating SupervisedTuningJob
SupervisedTuningJob created. Resource name: projects/768904645084/locations/us-central1/tuningJobs/8035538254927233024
To use this SupervisedTuningJob in another session:
tuning_job = sft.SupervisedTuningJob('projects/768904645084/locations/us-central1/tuningJobs/8035538254927233024')
View Tuning Job:
https://console.cloud.google.com/vertex-ai/generative/language/locations/us-central1/tuning/tuningJob/8035538254927233024?project=768904645084


KeyboardInterrupt: 

In [135]:
from google.cloud import aiplatform

# Vertex AI の初期化
#aiplatform.init(project="ai-agent-hackathon-447707", location="us-central1")

# デプロイ済みのエンドポイントを取得
endpoint = aiplatform.Endpoint(sft_tuning_job.tuned_model_endpoint_name)

print(f"Model deployed to: {endpoint.resource_name}")

ValueError: Resource  is not a valid resource id.

In [5]:
import vertexai
from vertexai.generative_models import GenerativeModel, SafetySetting, Part

def send_message(prompt):
    """Gemini API にメッセージを送信し、ボケやツッコミを生成"""
    vertexai.init(
        project="768904645084",
        location="us-central1",
        api_endpoint="us-central1-aiplatform.googleapis.com"
    )

    model = GenerativeModel(
        "projects/768904645084/locations/us-central1/endpoints/2971453263808823296",
    )

    chat = model.start_chat(response_validation=False)  # 変更点

    try:
        result = chat.send_message(
            [prompt],
            generation_config={
                "max_output_tokens": 50,  # 短文にする
                "temperature": 1,
                "top_p": 0.9,
            },
            safety_settings=[
                SafetySetting(
                    category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
                    threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH  # 厳しすぎるフィルタを緩和
                ),
                SafetySetting(
                    category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
                    threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH
                ),
                SafetySetting(
                    category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
                    threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH
                ),
                SafetySetting(
                    category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
                    threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH
                ),
            ]
        )
        return result
    except Exception as e:
        print(f"エラー発生: {e}")
        return ""

In [6]:
def assign_roles(comedians):
    """ボケとツッコミの役割を芸人の情報から決定"""
    boke = None
    tsukkomi = None

    if len(comedians) == 2:
        # `skills['role']` からボケとツッコミを決める
        for comedian in comedians:
            role = comedian.get('skills', {}).get('role', '')

            if 'ボケ' in role and boke is None:
                boke = comedian
            elif 'ツッコミ' in role and tsukkomi is None:
                tsukkomi = comedian

        # 万が一両方ボケ or 両方ツッコミなら、強制的に振り分け
        if not boke:
            boke = comedians[0]
        if not tsukkomi:
            tsukkomi = comedians[1]
    
    return boke, tsukkomi

In [7]:
def boke_agent(theme, context, geinin_info):
    """ボケエージェントが文脈と芸人情報を踏まえてボケを生成"""
    boke_prompt = f"""

    ## お題
    {theme}
    
    ## 芸人情報
    {geinin_info}

    ## 会話の流れ
    {context}

    あなたは漫才コンビのボケ担当です。
    以下のルールに必ず従って発言を行ってください：
    
    ・contextの一番最後の文章がツッコミです。そのフリ（ツッコミ）を受けてユーモアのあるボケを短く生成してください。
    ・長い説明は禁止。発言は長くとも2文まででまとめ、短くインパクトのある表現を心がけること。
    ・芸人情報と会話の流れを参考に回答を生成してください。
    ・生成したボケの文章だけを出力してください。
    """
    return send_message(boke_prompt)

def tsukkomi_agent(theme, context, geinin_info):
    """ツッコミエージェントが文脈と芸人情報を踏まえてツッコミを生成"""
    tsukkomi_prompt = f"""
    ## お題
    {theme}
    
    ## 芸人情報
    {geinin_info}

    ## 会話の流れ
    {context}

    あなたは漫才コンビのツッコミ担当です。
    以下のルールに従って発言を行ってください：
    
    ・会話の流れを踏まえて面白くツッコミを行い、その後に次のボケがしやすいフリを発言してください。
    ・ツッコミとフリはそれぞれ1文ずつで、短く簡潔にまとめてください。
    ・芸人情報と会話の流れを参考にしてください。
    ・生成したツッコミの文章以外を出力するのは禁止です。
    """
    return send_message(tsukkomi_prompt)

def first_tsukkomi_agent(theme, geinin_info):
    """ツッコミエージェントが文脈と芸人情報を踏まえてツッコミを生成"""
    tsukkomi_prompt = f"""
    ## お題
    {theme}
    
    ## 芸人情報
    {geinin_info}

    あなたは漫才コンビのツッコミ担当です。
    以下のルールに従って発言を行ってください：
    
    ・テーマに沿って次のボケがしやすいフリを発言してください。
    ・フリは1文で、短く簡潔にまとめてください。
    ・芸人情報と会話の流れを参考にしてください。
    ・生成したフリ以外の文章以外を出力するのは禁止です。
    """
    return send_message(tsukkomi_prompt)

In [8]:
def extract_text_from_response(response):
    """Gemini API のレスポンスからツッコミのテキストを抽出"""
    try:
        candidates = response.candidates
        if candidates:
            content = candidates[0].content
            if content:
                parts = content.parts
                if parts:
                    return parts[0].text
    except Exception as e:
        print(f"エラー発生: {e}")
    return ""

In [10]:
import time

comedians = get_random_comedians_data()
boke_info, tsukkomi_info = assign_roles(comedians)

context = ""
theme = "サブスク依存症"
if boke_info and tsukkomi_info:
    tsukkomi = first_tsukkomi_agent(theme, tsukkomi_info)
    tsukkomi_text = extract_text_from_response(tsukkomi)
    print("ツッコミ:", tsukkomi_text)
    context += f"\n1. ツッコミ: {tsukkomi_text}"
    
    for i in range(5):
        time.sleep(8)  # API 負荷を減らすために8秒待つ

        boke = boke_agent(theme, context, boke_info)
        boke_text = extract_text_from_response(boke)
        print(f"ボケ: {boke_text}\n")
        context += f"\n{i}. ボケ:{boke_text}"
        
        tsukkomi = tsukkomi_agent(theme, context, tsukkomi_info)
        tsukkomi_text = extract_text_from_response(tsukkomi)
        print(f"ツッコミ: {tsukkomi_text}\n")

        context += f"\n{i}. ツッコミ: {tsukkomi_text}"
    
    print("ツッコミ:　もうええわ。ありがとうございました。")

[{'gender': 'male', 'skills': {'writing_skill': 3, 'specialty_topics': 'フリースタイルラップ、モノマネ', 'voice_characteristics': '特に記載なし', 'role': 'ツッコミ'}, 'birthdate': '1987-01-26', 'agency': 'プロ（タイタン）', 'name': 'ヤマゲン'}, {'gender': 'male', 'skills': {'writing_skill': 3, 'specialty_topics': '大食い、サッカー・フットサル、ボウリング、ダーツ、スーパー銭湯、名探偵コナン', 'voice_characteristics': '低音ボイス', 'role': 'ツッコミ'}, 'birthdate': '1985-08-19', 'agency': 'プロ（吉本興業）', 'name': '益田康平'}]
ツッコミ: 最近、いろんなサブスクに登録しすぎちゃってさ。

ボケ: 解約の仕方がサブスクリプションされてる！


ツッコミ: 解約方法がサブスクってどういう状況だよ！で、結局いくつのサブスクに登録してるんだよ？


ボケ: 動画配信サービスだけでNetflix、Amazonプライム、Hulu、U-NEXT、Disney+、dアニメストア、ABEMAプレミアム、パラビ！全部で8個！「エイトマン」って呼ばれてる。


ツッコミ: エイトマンて！昭和のヒーローやん！そんなに観る時間あるの？


ボケ: 倍速再生だから余裕っすよ！1.5倍速だと「ジュウニマン」って呼ばれてるんすけどね。


ツッコミ: 十二マンて！倍速再生で名前変わるな！で、他にどんなサブスク入ってんの？


ボケ: あと、洋服レンタルのサブスク６つ入ってて、毎日違う服着てるんで「ムイマン」って呼ばれてるんすよ。


ツッコミ: ムイマンて！六つでムイマンはおかしいだろ！で、他に何か変わったサブスク入ってんの？


ボケ: あと、家事代行サービスのサブスク7つ入ってるんすよ。家事やりすぎて「ナナシ」って呼ばれてるんすけどね。


ツッコミ: ナナシって！七つでナナシは意味不明すぎるだろ！他に何か無駄なサブスク入ってないよね？


ツッコミ:　もうえ

In [13]:
def send_theme_prompt(prompt):
    """Gemini API にメッセージを送信し、ボケやツッコミを生成"""
    vertexai.init(
        project="768904645084",
        location="us-central1",
        api_endpoint="us-central1-aiplatform.googleapis.com"
    )

    model = GenerativeModel("gemini-1.5-pro")

    chat = model.start_chat(response_validation=False)  # 変更点

    try:
        result = chat.send_message(
            [prompt],
            generation_config={
                "max_output_tokens": 500,
                "temperature": 1,
                "top_p": 0.9,
            },
            safety_settings=[
                SafetySetting(
                    category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
                    threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH  # 厳しすぎるフィルタを緩和
                ),
                SafetySetting(
                    category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
                    threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH
                ),
                SafetySetting(
                    category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
                    threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH
                ),
                SafetySetting(
                    category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
                    threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH
                ),
            ]
        )
        return result
    except Exception as e:
        print(f"エラー発生: {e}")
        return ""

In [22]:
def create_theme():
    prompt = """
    #指令
    漫才のテーマを5つ考えて下さい。
    考えた5つのテーマをJSON形式で出力して下さい。
    テーマはお笑いの要素を含みつつ独自性のあるものを考えて下さい。

    #形式
    {
        "themes": [
            {
                "theme": "~~~"
                "description":"~~~"
            },
            {
                "theme": "~~~"
                "description":"~~~"
            },
            {
                "theme": "~~~"
                "description":"~~~"
            },
            {
                "theme": "~~~"
                "description":"~~~"
            },
            {
                "theme": "~~~"
                "description":"~~~"
            }
        ]
    }
    """
    response = send_theme_prompt(prompt)

    return response

In [23]:
theme = create_theme()

In [28]:
import json
theme_text = theme.candidates[0].content.parts[0].text
print(theme_text)

# 文字列として取得した JSON を Python の辞書に変換
theme_data = json.loads(theme_text)

# "themes" リストから "theme" だけを抽出
theme_list = [item["theme"] for item in theme_data["themes"]]

# 結果を表示
print(theme_list)

{
        "themes": [
            {
                "theme": "未来のコンビニ",
                "description":"近未来、テクノロジーがさらに進化したコンビニでのボケとツッコミ。ロボット店員、ドローン配達、進化した商品など、想像力を活かせる。"
            },
            {
                "theme": "幽霊の就職活動",
                "theme":"この世に未練を残した幽霊が就職活動をするという設定。履歴書の書き方、面接対策、希望職種など、ブラックユーモアを交えながら展開できる。"
            },
            {
                "theme": "AIスピーカーがツッコミ担当",
                "description":"ボケ担当の人間と、冷静沈着なAIスピーカーの漫才。最新の情報やデータを取り入れたツッコミで、ボケを論破していく。"
            },
            {
                "theme": "歴史上の人物によるYouTuber",
                "description":"もしも歴史上の人物がYouTuberだったら？織田信長、クレオパトラ、ナポレオンなど、誰もが知る人物が現代風の動画配信をする面白さ。"
            },
            {
                "theme": "動物語翻訳機",
                "description":"動物語翻訳機を使ってペットの本音を聞いてみたものの、予想外の言葉が返ってくるという設定。ペットの可愛いらしさと、予想外の言葉のギャップで笑いを誘う。"
            }
        ]
}
['未来のコンビニ', 'この世に未練を残した幽霊が就職活動をするという設定。履歴書の書き方、面接対策、希望職種など、ブラックユーモアを交えながら展開できる。', 'AIスピーカーがツッコミ担当', '歴史上の人物によるYouTuber', '動物語翻訳機']
